## Data process v2
This script is used for train/test/valid dataset generation for GNHS related data.  
Phase 1 patients data is split into train/test(9/1) where phase 4 patients data is used for validation.

NB. the difference between version 1 and version 2 is version 2 will not consider any risk factor of mets 

In [2]:
import os
import sys

os.chdir("..")
sys.path.append(r"D:\GitHub\GNHS\src")

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from utils.utils import sub_label_generator
from models.feature_selection import distribution_deviation
import snf
pd.set_option("display.max_columns", None)

seed = 0
np.random.seed(seed)

In [4]:
root_dir = Path(r"D:\GitHub\GNHS").resolve()

# data baseline
data_baseline = root_dir / "data/raw_data/20220413_discovery_machinelearning_delriskfactor.csv" 
data_perspective = root_dir/ "data/raw_data/20220413_validation_machinelearning_delriskfactor.csv"

data_baseline_all = root_dir / "data/raw_data/20220322_discovery_machinelearning.csv" 
data_perspective_all = root_dir/ "data/raw_data/20220322_validation_machinelearning.csv"

# output directory
TRANSFORM = root_dir/ "data/transformed_data/2022"

In [5]:
# process data with all features
excluding = ["ID", "time", "metsscore"]
clinic_list = ["age", "BMI", "DBP", "Glu", "HDL", "LDL", "SBP", "TG", "wc",	"TC", "smoke", "alc", "tea", "height", "weight", "hipc"]
df_baseline = pd.read_csv(data_baseline_all, index_col="match")
df_perspective = pd.read_csv(data_perspective_all, index_col="match")

df_baseline = df_baseline.loc[:, ~df_baseline.columns.isin(excluding)]
df_perspective = df_perspective.loc[:, ~df_perspective.columns.isin(excluding)]

df_baseline_prot = df_baseline.loc[:, ~df_baseline.columns.isin(clinic_list)]
df_baseline_clinic = df_baseline.loc[:, df_baseline.columns.isin(clinic_list + ["label"])]

df_perspective_prot = df_perspective.loc[:, ~df_perspective.columns.isin(clinic_list)]
df_perspective_clinic = df_perspective.loc[:, df_perspective.columns.isin(clinic_list + ["label"])]

df_baseline_prot.shape, df_baseline_clinic.shape, df_perspective_prot.shape, df_perspective_clinic.shape

((855, 439), (855, 17), (242, 416), (242, 17))

In [6]:
# filter out proteins with more than thershold NA rate
threshold = 0.3
df_baseline_prot = df_baseline_prot.loc[:, (df_baseline_prot.isna().sum(axis=0) / df_baseline_prot.shape[0]) <= threshold]
protein_list = df_baseline_prot.columns.to_list()
extra_protein = list(set(df_baseline_prot.columns) - set(df_perspective_prot.columns))
df_perspective_prot[extra_protein] = 0
df_perspective[extra_protein] = 0
print(extra_protein)
df_perspective_prot = df_perspective_prot[protein_list]
print(df_baseline_prot.shape, df_perspective_prot.shape)
len(set(df_baseline_prot.columns).intersection(set(df_perspective_prot.columns)))

['O60667_FCMR', 'Q5HYC2_KIAA2026']
(855, 289) (242, 289)



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


289

In [9]:
df_perspective_prot.columns.to_list()

['A0A075B6I0_IGLV8.61',
 'A0A075B6Q5_IGHV3.64',
 'A0A087WSY6_IGKV3D.15',
 'A0A0A0MS15_IGHV3.49',
 'A0A0B4J1U7_IGHV6.1',
 'A0A0B4J1V0_IGHV3.15',
 'A0A0B4J1X5_IGHV3.74',
 'A0A0B4J1X8_IGHV3.43',
 'A0A0B4J1Y9_IGHV3.72',
 'A0A0C4DH25_IGKV3D.20',
 'A0A0C4DH34_IGHV4.28',
 'A0A0C4DH35',
 'A0A0C4DH36',
 'A0A0C4DH42_IGHV3.66',
 'A0A0C4DH68_IGKV2.24',
 'A0A0C4DH69_IGKV1.9',
 'A0A0J9YVY3_IGHV7.4.1',
 'A0A0J9YX35_IGHV3.64D',
 'A6NFN9_ANKUB1',
 'A6NGW2_STRCP1',
 'A6NMB9_FIGNL2',
 'A6NNL5_C15orf61',
 'B4DU55_ZNF879',
 'O00748_CES2',
 'O14559_ARHGAP33',
 'O14791_APOL1',
 'O43866_CD5L',
 'O60667_FCMR',
 'O60890_OPHN1',
 'O60909_B4GALT2',
 'O75636_FCN3',
 'O75882_ATRN',
 'O94915_FRYL',
 'O94989_ARHGEF15',
 'O95445_APOM',
 'O95714_HERC2',
 'O95954_FTCD',
 'P00450_CP',
 'P00734_F2',
 'P00736_C1R',
 'P00738_HP',
 'P00739_HPR',
 'P00740_F9',
 'P00742_F10',
 'P00746_CFD',
 'P00747_PLG',
 'P00748_F12',
 'P00751_CFB',
 'P01008_SERPINC1',
 'P01009_SERPINA1',
 'P01011_SERPINA3',
 'P01019_AGT',
 'P01023_A2M',
 'P

In [7]:
df_baseline.to_csv(TRANSFORM / "mets/mets_combined_baseline_full.csv")
df_baseline_prot.to_csv(TRANSFORM / "mets/mets_protein_baseline_full.csv")
df_baseline_clinic.to_csv(TRANSFORM / "mets/mets_clinic_baseline_full.csv")

df_perspective.to_csv(TRANSFORM / "mets/mets_combined_perspective_full.csv")
df_perspective_prot.to_csv(TRANSFORM / "mets/mets_protein_perspective_full.csv")
df_perspective_clinic.to_csv(TRANSFORM / "mets/mets_clinic_perspective_full.csv")

In [8]:
df_discovery = pd.read_csv(data_baseline, index_col="match")
df_validation = pd.read_csv(data_perspective, index_col="match")

df_discovery_prot = df_discovery.loc[:, ~df_discovery.columns.isin(["age", "sex"])]
df_discovery_clinic = df_discovery.loc[:, df_discovery.columns.isin(["age", "sex", "label"])]

df_validation_prot = df_validation.loc[:, ~df_validation.columns.isin(["age", "sex"])]
df_validation_clinic = df_validation.loc[:, df_validation.columns.isin(["age", "sex", "label"])]

df_discovery_prot.shape, df_discovery_clinic.shape, df_validation_prot.shape, df_validation_clinic.shape

((855, 439), (855, 3), (242, 414), (242, 3))

In [9]:
# filter out proteins with more than thershold NA rate
threshold = 0.3
df_discovery_prot = df_discovery_prot.loc[:, (df_discovery_prot.isna().sum(axis=0) / df_discovery_prot.shape[0]) <= threshold]
protein_list = df_discovery_prot.columns.to_list()
extra_protein = list(set(df_discovery_prot.columns) - set(df_validation_prot.columns))
df_validation_prot[extra_protein] = 0
# print(extra_protein)
df_validation_prot = df_validation_prot[protein_list]
print(df_discovery_prot.shape, df_validation_prot.shape)
len(set(df_discovery_prot.columns).intersection(set(df_validation_prot.columns)))

(855, 289) (242, 289)



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


289

In [45]:
df_discovery.to_csv(TRANSFORM / "mets/mets_combined_baseline_delriskfactor.csv")
df_discovery_prot.to_csv(TRANSFORM / "mets/mets_protein_baseline_delriskfactor.csv")
df_discovery_clinic.to_csv(TRANSFORM / "mets/mets_clinic_baseline_delriskfactor.csv")

df_validation.to_csv(TRANSFORM / "mets/mets_combined_perspective_delriskfactor.csv")
df_validation_prot.to_csv(TRANSFORM / "mets/mets_protein_perspective_delriskfactor.csv")
df_validation_clinic.to_csv(TRANSFORM / "mets/mets_clinic_perspective_delriskfactor.csv")

In [11]:
df_discovery_prot

A0A075B6I0_IGLV8.61  A0A075B6Q5_IGHV3.64  \
match                                                      
NL1002baseline             4.496423             7.700191   
NL1006baseline             5.719225             7.812774   
NL1010baseline             7.047653             7.256793   
NL1012baseline             6.893794             7.292500   
NL1013baseline             6.816304             7.547338   
...                             ...                  ...   
NL948baseline              6.475502             6.593062   
NL953baseline              6.568001                  NaN   
NL982baseline              6.957472             7.729748   
NL992baseline              6.667529             7.715123   
NL998baseline              6.677507             7.333203   

                A0A087WSY6_IGKV3D.15  A0A0A0MS15_IGHV3.49  A0A0B4J1U7_IGHV6.1  \
match                                                                           
NL1002baseline              6.365705             7.461191            8.120980   
NL1006baseline              6.674887             8.456629            7.779244   
NL1010baseline              6.066927             7.892787            8.693063   
NL1012baseline              6.335874             7.634618            8.185976   
NL1013baseline              5.870841             7.013595            8.302698   
...                              ...                  ...                 ...   
NL948baseline               5.809015             7.730180            9.159883   
NL953baseline               6.014642             7.435977            7.759477   
NL982baseline               5.795359             6.815185            8.943757   
NL992baseline               6.922279             7.319949            8.110534   
NL998baseline               6.634453             7.747093            9.061422   

                A0A0B4J1V0_IGHV3.15  A0A0B4J1X5_IGHV3.74  A0A0B4J1X8_IGHV3.43  \
match                                                                           
NL1002baseline             2.894668             8.481862             4.538502   
NL1006baseline             3.900700             9.630911             5.936362   
NL1010baseline             4.840423             8.699242             5.206834   
NL1012baseline             3.933514             8.851103             6.042372   
NL1013baseline             3.292805             9.131615             4.792398   
...                             ...                  ...                  ...   
NL948baseline                   NaN             8.708165             5.285779   
NL953baseline              4.072442             8.459453             4.509116   
NL982baseline              3.231649             8.694756             5.218610   
NL992baseline              5.270321             9.352354             4.899318   
NL998baseline              3.573108             8.857739             6.061617   

                A0A0B4J1Y9_IGHV3.72  A0A0C4DH25_IGKV3D.20  \
match                                                       
NL1002baseline                  NaN              7.565303   
NL1006baseline             8.765701              7.974182   
NL1010baseline             8.934696              7.090424   
NL1012baseline             8.415499              8.068730   
NL1013baseline             8.127022              7.268222   
...                             ...                   ...   
NL948baseline              8.026331              7.375903   
NL953baseline              8.882777              7.266297   
NL982baseline              8.444774              7.291200   
NL992baseline              8.623874              8.710888   
NL998baseline              8.055920              7.209656   

                A0A0C4DH34_IGHV4.28  A0A0C4DH35  A0A0C4DH36  \
match                                                         
NL1002baseline             4.974479    6.987709    6.604089   
NL1006baseline             5.352653    7.680299    7.192482   
NL1010baseline             4.829396    7.264331    6.533555   
NL1012baseline    